<a href="https://colab.research.google.com/github/Ronaldo-khagokpam/DEEP-LEARNING/blob/main/cnn3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch as F
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Define the CNN model
class CNN(nn.Module):
    def _init_(self):
        super(CNN, self)._init_()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self,trainset):
        trainset = self.pool(F.relu(self.conv1(trainset)))
        trainset = self.pool(F.relu(self.conv2(trainset)))
        trainset = trainset.view(-1, 16*5*5)
        trainset = F.relu(self.fc1(trainset))
        trainset = F.relu(self.fc2(trainset))
        trainset = self.fc3(trainset)
        return trainset

# Set the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the Cifar10 dataset
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

# Initialize the CNN model and optimizer
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# Train the model
for epoch in range(20):
    for x, y in trainloader:

        x, y = torch.Tensor(x).to(device), torch.Tensor(y).to(device)
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluate the model
model.eval()
test_loss = 0
correct = 0
y_pred = []
y_true = []
with torch.no_grad():
    for x, y in testloader:
        x, y = torch.Tensor(x).to(device), torch.Tensor(y).to(device)
        output = model(x)
        loss = criterion(output, y)
        test_loss += loss.item()
        _, predicted = torch.max(output, 1)
        correct += (predicted == y).sum().item()
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(y.cpu().numpy())

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
conf_mat = confusion_matrix(y_true, y_pred)

print(f'Test Loss: {test_loss / len(testloader)}')
print(f'Test Accuracy: {accuracy:.2f}%')
print(f'Test Precision: {precision:.2f}%')
print(f'Test Recall: {recall:.2f}%')
print(f'Test F1 Score: {f1:.2f}%')
print(f'Confusion Matrix: \n{conf_mat}')